In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import psycopg2
from bs4 import BeautifulSoup as bp
import time
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from shapely.geometry import Point
import requests
import csv


In [ ]:
import getpass
import os

postgres_url=getpass.getpass("enter your postgresql url: ")


In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
REQUEST_HEADER = {
    'User-Agent': USER_AGENT,
    'Accept-language': 'vi-vn',
}

In [3]:
options=Options()
options.add_argument(f"user-agent={USER_AGENT}")
driver=webdriver.Chrome(options=options)

In [6]:
driver.get('https://www.google.com/maps')

In [4]:
def get_page_html(url):
    res = requests.get(url=url, headers=REQUEST_HEADER)
    return res.text
def get_hotel_name(soup):
    name = soup.find('div', class_='css-901oao r-a5wbuh r-1enofrn r-b88u0q r-1cwl3u0 r-fdjqy7 r-3s2u2q')
    return name.text.strip() if name else None
def get_hotel_address(soup,name):
    address = soup.find('div', class_='css-901oao css-cens5h r-13awgt0 r-a5wbuh r-1b43r93 r-majxgm r-rjixqe r-fdjqy7')
    if address:
        
        address_full = address.text.strip().replace('\t', '')
        return address_full
    else:
        print(f'cannot find address for {name}')


In [ ]:
def get_hotel_id(name):
    query="""
        SELECT hotel_id
        FROM hotel
        WHERE name ILIKE %s;"""
    conn = psycopg2.connect(postgres_url)
    cur = conn.cursor()
    cur.execute("""SET search_path TO travel_database, public;""")
    cur.execute(query,(name,))
    row=cur.fetchone()
    hotel_id = row[0]
    cur.close()
    return hotel_id

In [ ]:
def insert_db_hotel_price(price,id):
    print(f"insert price into db of room {id}")
    conn = psycopg2.connect(postgres_url)
    cur = conn.cursor()
    cur.execute("""SET search_path TO travel_database, public;""")
    query="""
    UPDATE Hotelprice SET price = %s
    WHERE hotel_id = %s;"""
    cur.execute(query, (price,id))
    conn.commit()
    cur.close()
    conn.close()

In [5]:
def get_location(address):
    # Click the search box input element
    print(f'get location for {address}')
    find = driver.find_element(By.XPATH, '//*[@id="searchboxinput"]')
    find.click()
    find.clear()
    time.sleep(2)

# Input the address
    find.send_keys(address)
    time.sleep(2)
    button=driver.find_element(By.XPATH,'//*[@id="searchbox-searchbutton"]')
    button.click()
    time.sleep(5)
    current_url=driver.current_url
    current_url=current_url.split('/')
    current_url
    raw_location=current_url[6].split(',')
    raw_location
    latitude_float=float(raw_location[0].replace('@',''))
    longitude_float=float(raw_location[1])
# In ra kết quả hoặc lưu vào biến
    print(f"Latitude: {latitude_float}, Longitude: {longitude_float}")
    return {'latitude': latitude_float, 'longitude': longitude_float}

In [ ]:
def insert_location(id,location):
    print(f"insert location {id} into db")
    if isinstance(location, dict) and 'longitude' in location and 'latitude' in location:
        point = Point(location['longitude'], location['latitude'])
        point_wkt = point.wkt
        conn = psycopg2.connect(postgres_url)
        cur = conn.cursor()
        cur.execute("""SET search_path TO travel_database, public;""")
        query="""
        UPDATE Hotel SET location = ST_GeomFromText(%s, 4326)
        WHERE hotel_id = %s;"""
        cur.execute(query, (point_wkt,id))
        conn.commit()
        cur.close()
        conn.close()
        print(f"updated location for {id}")
    else:
        print("error: location is not dict")

In [10]:
for i in range(1,2):
    print(f'hotel_urls_{i}:')
    with open(f'D:/random_code/hotel_urls/hotel_urls/hotel_urls_{i}.csv', newline='',encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        j=1
        for row in reader:
            print(f'scraping url: {j}')
            url=row[0]
            html=get_page_html(url)
            time.sleep(0.5)
            soup=bp(html,'lxml')
            print('HTML fetched successful')
            name=get_hotel_name(soup)
            id=get_hotel_id(name)
            add=get_hotel_address(soup,name)
            time.sleep(0.5)
            location=get_location(add)
            insert_location(id,location)
            time.sleep(2)
            j+=1

hotel_urls_1:
scraping url: 1
HTML fetched successful
get location for 3 B9 Đầm Trấu, Bạch Đằng, Quận Hai Bà Trưng, Hà Nội, Việt Nam, 11600
Latitude: 21.0120439, Longitude: 105.8651544
scraping url: 1
HTML fetched successful
get location for 21 Hàng Cá, Hàng Đào, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 101113
Latitude: 21.0360747, Longitude: 105.8487235
scraping url: 1
HTML fetched successful
get location for 14 P. Lý Nam Đế, Quán Thánh, Hoàn Kiếm, Hàng Mã, Quận Hoàn Kiếm, Hà Nội, Việt Nam, 100000


KeyboardInterrupt: 